<a href="https://colab.research.google.com/github/Dimildizio/DS_course/blob/main/Neural_networks/NLP/Langchain/Mistral/Mistral_RAG_function_calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic RAG (Retrieval Augmented Generation)

In [ ]:
! pip install faiss-cpu "mistralai>=0.1.2"

### Load API key

In [ ]:
from helper import load_mistral_api_key
api_key, dlai_endpoint = load_mistral_api_key(ret_key=True)

### Get data

- You can go to https://www.deeplearning.ai/the-batch/
- Search for any article and copy its URL.

### Parse the article with BeautifulSoup

In [ ]:
import requests
from bs4 import BeautifulSoup
import re

response = requests.get(
    "https://www.deeplearning.ai/the-batch/24-7-phish-fry/"
)
html_doc = response.text
soup = BeautifulSoup(html_doc, "html.parser")
tag = soup.find("div", re.compile("^prose--styled"))
text = tag.text
print(text)

Foiling attackers who try to lure email users into clicking on a malicious link is a cat-and-mouse game, as phishing tactics evolve to evade detection. But machine learning models designed to recognize phishing attempts can evolve, too, through automatic retraining and checks to maintain accuracy.What’s new: Food giant Nestlé built a system that checks incoming emails and sends suspicious ones to the company’s security team. Microsoft’s Azure Machine Learning web platform supplies the tools and processing power.Problem: Nestlé receives up to 20 million emails in 300,000 inboxes daily. An earlier commercial system flooded analysts with legitimate messages wrongly flagged as phishing attempts — too many to evaluate manually.Solution: The company built an automated machine learning system that continually learns from phishing attempts, spots likely new ones, and forwards them to security analysts.How it works: The system comprises three automated pipelines that run in the cloud. The first

### Optionally, save the text into a text file
- You can upload the text file into a chat interface in the next lesson.
- To download this file to your own machine, click on the "Jupyter" logo to view the file directory.  

In [ ]:
file_name = "AI.txt"
with open(file_name, 'w') as file:
    file.write(text)

### Chunking

In [ ]:
chunk_size = 512
chunks = [text[i : i + chunk_size] for i in range(0, len(text), chunk_size)]

In [ ]:
len(chunks)

6

### Get embeddings of the chunks

In [ ]:
import os
from mistralai.client import MistralClient


def get_text_embedding(txt):
    client = MistralClient(api_key=api_key, endpoint=dlai_endpoint)
    embeddings_batch_response = client.embeddings(model="mistral-embed", input=txt)
    return embeddings_batch_response.data[0].embedding

In [ ]:
import numpy as np

text_embeddings = np.array([get_text_embedding(chunk) for chunk in chunks])

In [ ]:
text_embeddings

array([[-0.0075531 ,  0.02249146,  0.05093384, ..., -0.01940918,
         0.01558685, -0.02246094],
       [-0.01600647,  0.02281189,  0.04702759, ..., -0.03353882,
         0.01012421, -0.03091431],
       [ 0.00303078,  0.02993774,  0.0289917 , ..., -0.0211792 ,
         0.0141449 , -0.04074097],
       [-0.03805542,  0.03686523,  0.0324707 , ..., -0.0221405 ,
         0.00184155, -0.01629639],
       [-0.0206604 ,  0.03085327,  0.0534668 , ..., -0.01698303,
         0.00089979, -0.02096558],
       [-0.02661133,  0.02787781,  0.03442383, ..., -0.01727295,
         0.02000427,  0.00232124]])

In [ ]:
len(text_embeddings[0])

1024

### Store in a vector databsae
- In this classroom, you'll use [Faiss](https://engineering.fb.com/2017/03/29/data-infrastructure/faiss-a-library-for-efficient-similarity-search/)

In [ ]:
import faiss

d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

### Embed the user query

In [ ]:
question = "How does the new phishing classification system work?"
question_embeddings = np.array([get_text_embedding(question)])

In [ ]:
question_embeddings

array([[ 0.02009583,  0.02420044,  0.03068542, ..., -0.02127075,
        -0.00865173, -0.01152039]])

### Search for chunks that are similar to the query

In [ ]:
D, I = index.search(question_embeddings, k=2)
print(I)

[[2 0]]


In [ ]:
retrieved_chunk = [chunks[i] for i in I.tolist()[0]]
print(retrieved_chunk)

['econd evaluates incoming messages, and the third passes the latest risky messages to security.The system stores incoming emails in a data lake. A transformer model fine-tuned for the task examines email subject headers to classify phishing attempts. Factors such as the sender’s domain are used to prioritize messages for human attention.It processes incoming messages hourly in batches that run in parallel and retrains the model weekly to learn from the latest attacks. It also retrains the model whenever the ', 'Foiling attackers who try to lure email users into clicking on a malicious link is a cat-and-mouse game, as phishing tactics evolve to evade detection. But machine learning models designed to recognize phishing attempts can evolve, too, through automatic retraining and checks to maintain accuracy.What’s new: Food giant Nestlé built a system that checks incoming emails and sends suspicious ones to the company’s security team. Microsoft’s Azure Machine Learning web platform suppl

In [ ]:
prompt = f"""
Context information is below.
---------------------
{retrieved_chunk}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""

In [ ]:
from mistralai.models.chat_completion import ChatMessage


def mistral(user_message, model="mistral-small-latest", is_json=False):
    client = MistralClient(api_key=api_key, endpoint=dlai_endpoint)
    messages = [ChatMessage(role="user", content=user_message)]

    if is_json:
        chat_response = client.chat(
            model=model, messages=messages, response_format={"type": "json_object"}
        )
    else:
        chat_response = client.chat(model=model, messages=messages)

    return chat_response.choices[0].message.content

In [ ]:
response = mistral(prompt)
print(response)

The new phishing classification system works by evaluating incoming messages using a fine-tuned transformer model. This model examines email subject headers to classify potential phishing attempts. Factors such as the sender's domain are used to prioritize messages for further human attention. The system processes incoming messages hourly in batches that run in parallel. To stay updated with the latest phishing tactics, the model is retrained weekly and also whenever necessary. Suspicious messages are then passed on to the company's security team. This system is built on Microsoft's Azure Machine Learning web platform.


## RAG + Function calling

In [ ]:
def qa_with_context(text, question, chunk_size=512):
    # split document into chunks
    chunks = [text[i : i + chunk_size] for i in range(0, len(text), chunk_size)]
    # load into a vector database
    text_embeddings = np.array([get_text_embedding(chunk) for chunk in chunks])
    d = text_embeddings.shape[1]
    index = faiss.IndexFlatL2(d)
    index.add(text_embeddings)
    # create embeddings for a question
    question_embeddings = np.array([get_text_embedding(question)])
    # retrieve similar chunks from the vector database
    D, I = index.search(question_embeddings, k=2)
    retrieved_chunk = [chunks[i] for i in I.tolist()[0]]
    # generate response based on the retrieve relevant text chunks

    prompt = f"""
    Context information is below.
    ---------------------
    {retrieved_chunk}
    ---------------------
    Given the context information and not prior knowledge, answer the query.
    Query: {question}
    Answer:
    """
    response = mistral(prompt)
    return response

In [ ]:
I.tolist()

[[2, 0]]

In [ ]:
I.tolist()[0]

[2, 0]

In [ ]:
import functools

names_to_functions = {"qa_with_context": functools.partial(qa_with_context, text=text)}

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "qa_with_context",
            "description": "Answer user question by retrieving relevant context",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {
                        "type": "string",
                        "description": "user question",
                    }
                },
                "required": ["question"],
            },
        },
    },
]

In [ ]:
question = "How does the new phishing classification system work?"
client = MistralClient(api_key=api_key, endpoint=dlai_endpoint)

response = client.chat(
    model="mistral-large-latest",
    messages=[ChatMessage(role="user", content=question)],
    tools=tools,
    tool_choice="auto",
)

response

ChatCompletionResponse(id='87204b5efa7e43b99c9571b8e3ba30f5', object='chat.completion', created=1722697303, model='mistral-large-latest', choices=[ChatCompletionResponseChoice(index=0, message=ChatMessage(role='assistant', content='', name=None, tool_calls=[ToolCall(id='5PZbEpA5D', type=<ToolType.function: 'function'>, function=FunctionCall(name='qa_with_context', arguments='{"question": "How does the new phishing classification system work?"}'))]), finish_reason=<FinishReason.tool_calls: 'tool_calls'>)], usage=UsageInfo(prompt_tokens=86, total_tokens=117, completion_tokens=31))

In [ ]:
tool_function = response.choices[0].message.tool_calls[0].function
tool_function

FunctionCall(name='qa_with_context', arguments='{"question": "How does the new phishing classification system work?"}')

In [ ]:
tool_function.name

'qa_with_context'

In [ ]:
import json

args = json.loads(tool_function.arguments)
args

{'question': 'How does the new phishing classification system work?'}

In [ ]:
function_result = names_to_functions[tool_function.name](**args)
function_result

"The new phishing classification system works by storing incoming emails in a data lake and using a transformer model that has been fine-tuned for the task to examine email subject headers to classify potential phishing attempts. The system prioritizes messages for human attention based on factors such as the sender's domain. It processes incoming messages in parallel hourly batches and retrains the model weekly to learn from the latest attacks. Additionally, it sends the latest risky messages to the company's security team and can also retrain the model whenever necessary to maintain accuracy."

## More about RAG
To learn about more advanced chunking and retrieval methods, you can check out:
- [Advanced Retrieval for AI with Chroma](https://learn.deeplearning.ai/courses/advanced-retrieval-for-ai/lesson/1/introduction)
  - Sentence window retrieval
  - Auto-merge retrieval
- [Building and Evaluating Advanced RAG Applications](https://learn.deeplearning.ai/courses/building-evaluating-advanced-rag)
  - Query Expansion
  - Cross-encoder reranking
  - Training and utilizing Embedding Adapters
